In [ ]:

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command,interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    value: str

memory = MemorySaver()
def node_a(state: State): 
    print("Node A")
    return Command(
        goto="node_b", 
        update={
            "value": state["value"] + "a"
        }
    )

def node_b(state: State): 
    print("Node B")

    human_response = interrupt("Do you want to go to C or D? Type C/D")

    print("Human Review Values: ", human_response)
    
    if(human_response == "C"): 
        return Command(
            goto="node_c", 
            update={
                "value": state["value"] + "b"
            }
        ) 
    elif(human_response == "D"): 
        return Command(
            goto="node_d", 
            update={
                "value": state["value"] + "b"
            }
        )
    else:
        return Command(goto="cannot_answer", update={"value": state["value"] + "?"})


def node_c(state: State): 
    print("Node C")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "c"
        }
    )

def node_d(state: State): 
    print("Node D")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "d"
        }
)
def cannot_answer(state: State): 
    print("Invalid answer. Ending the flow with fallback.")
    return Command(goto=END, update={"value": state["value"] + "invalid"})

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)

graph.set_entry_point("node_a")


app = graph.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}
initialState = {
    "value": ""
}

first_result = app.invoke(initialState, config, stream_mode="updates")
first_result

Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:8b4ff212-6fc4-8fe6-c779-f5b2cd0a1295']),)}]

In [13]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from typing import TypedDict, Literal
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    value: str

memory = MemorySaver()

def node_a(state: State): 
    print("Node A")
    return Command(
        goto="node_b",
        update={"value": state["value"] + "a"}
    )

def node_b(state: State): 
    print("Node B")
    human_response = interrupt("Do you want to go to C or D? Type C/D").strip().upper()
    print("Human Response:", human_response)

    if human_response == "C": 
        return Command(goto="node_c", update={"value": state["value"] + "b"})
    elif human_response == "D": 
        return Command(goto="node_d", update={"value": state["value"] + "b"})
    else:
        return Command(goto="cannot_answer", update={"value": state["value"] + "?"})

def node_c(state: State): 
    print("Node C")
    return Command(goto=END, update={"value": state["value"] + "c"})

def node_d(state: State): 
    print("Node D")
    return Command(goto=END, update={"value": state["value"] + "d"})

def cannot_answer(state: State): 
    print("Invalid answer. Ending the flow with fallback.")
    return Command(goto=END, update={"value": state["value"] + "invalid"})

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)
graph.add_node("cannot_answer", cannot_answer)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}
initialState = {"value": ""}

result = app.invoke(initialState, config, stream_mode="updates")
result


Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:968ec58b-c619-9dbe-17e3-fbb288cb13f2']),)}]

In [14]:
print(app.get_state(config).next)

('node_b',)


In [15]:
second_result = app.invoke(Command(resume="D"), config=config, stream_mode="updates")
second_result

Task node_d with path ('__pregel_pull', 'node_d') wrote to unknown channel branch:to:__end__, ignoring it.


Node B
Human Response: D
Node D


[{'node_b': {'value': 'ab'}}, {'node_d': {'value': 'abd'}}]